## Setup

In [2]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
from scipy.spatial.distance import cdist
from google.colab.patches import cv2_imshow

In [49]:
from os import listdir
from os.path import isfile, join
import glob

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# # Defining constant variables
# YOLOV7_FOLDER = '/content/drive/My Drive/Bundesliga/YOLO7'
# VIDEOS_FOLDER = '/yolov7/runs/detect'
# CLIPS_FOLDER = YOLOV7_FOLDER + VIDEOS_FOLDER

In [58]:
POSITIVES_FOLDER = '/content/drive/MyDrive/Bundesliga/PositiveFrames/'
FOLDER_FRAMES = 'frames_ms2_1606b0e6_0/'
FOLDER_LABELS = 'labels/'
os.chdir(POSITIVES_FOLDER)

In [5]:
! pwd

/content/drive/MyDrive/Bundesliga/PositiveFrames


## To modify

In [32]:
video = '1606b0e6_0' # I don't need this

## Cropping original frames

In [7]:
CLASS_INDEX = 0
X_INDEX = 1
Y_INDEX = 2
PERSON_CLASS = '0'
BALL_CLASS = '32'
X_SIZE = 1928
Y_SIZE = 1024
CROP_SIZE = 224 # pixels, size of crop = (CROP_SIZExCROP_SIZE)
radius = int(CROP_SIZE / 2)

In [8]:
! pwd

/content/drive/MyDrive/Bundesliga/PositiveFrames


In [9]:
# files = [file for file in listdir(FOLDER_FRAMES) if isfile(join(FOLDER_FRAMES, file))] # get a list of files in the directory
# filenames = [os.path.splitext(filename)[0] for filename in files] # separate filenames from their extension .jpg

In [10]:
# files[1]

## Test: doing above by iterating over 2 different folders (frames and labels)

In [81]:
POSITIVES_FOLDER = '/content/drive/MyDrive/Bundesliga/PositiveFrames/'
FOLDER_LABELS = POSITIVES_FOLDER + 'detect/'
FOLDER_FRAMES = POSITIVES_FOLDER + 'frames_ms2/'

FIRST_FRAMES = FOLDER_FRAMES + '1606b0e6_0/'
SECOND_FRAMES = FOLDER_FRAMES + '1606b0e6_1/'

In [12]:
# CURRENT_FOLDER = FIRST_FRAMES # TO CHANGE
# os.chdir(CURRENT_FOLDER)
# ! pwd

In [33]:
video

'1606b0e6_0'

In [18]:
! ls $video | wc -l

13860


In [19]:
# CURRENT_FOLDER = SECOND_FRAMES # TO CHANGE
# os.chdir(CURRENT_FOLDER)
# ! pwd

In [199]:
# POSITIVES_FOLDER = '/content/drive/MyDrive/Bundesliga/PositiveFrames/'
# FOLDER_LABELS = POSITIVES_FOLDER + 'detect/'
# FOLDER_FRAMES = POSITIVES_FOLDER + '1606b0e6_0/'

In [21]:
# files = [file for file in listdir(FIRST_FRAMES) if isfile(join(FIRST_FRAMES, file))] # get a list of files in the directory
# filenames = [os.path.splitext(filename)[0] for filename in files] # separate filenames from their extension .jpg

In [22]:
# sum('1606b0e6_0_11' in(file) for file in files)

In [23]:
# sum('1606b0e6_0_24' in(file) for file in files)

In [24]:
# filenames[-5:]

## Main code:

In [53]:
ANY_INDEX = 0

In [25]:
# video = "_".join(filenames[ANY_INDEX].split("_", 2)[:2])
# rest = ''.join(filenames[ANY_INDEX].split("_", 2)[2:])
# y = filenames[ANY_INDEX].split("_")[3]
# video, rest, y

In [26]:
FOLDER_LABELS

'/content/drive/MyDrive/Bundesliga/PositiveFrames/detect/'

In [27]:
current_labels = FOLDER_LABELS + video + '/labels/'
current_labels

'/content/drive/MyDrive/Bundesliga/PositiveFrames/detect/1606b0e6_0//labels/'

In [28]:
USE_PERSONS = True
USE_PERSONS_THRESH = 15
FRAMES_PER_EVENT = 35
NUM_CLASSES = 3
CONFIDENCE_INDEX = 5

In [34]:
video

'1606b0e6_0'

In [35]:
EVENTS_PER_VIDEO = {'1606b0e6_0': 396} # MAYBE CODE IT IN A BETTER WAY

In [36]:
EVENTS_PER_VIDEO[video]

396

In [37]:
FRAMES_FORMAT = 'VIDEOID_EVENTID_CLASSID_ORDERID.jpg'

In [38]:
class_meanings = {0: 'challenge', 1: 'throw-in', 2: 'play'}

In [39]:
# Challenges' frame between tolerances are respectively: 17 yes, 1 yes, 17 yes.
# For play and throw-in are respectively: 9 no, 8 yes, 1 yes, 8 yes, 9 no.

Implementation:

In [82]:
POSITIVES_FOLDER = '/content/drive/MyDrive/Bundesliga/PositiveFrames/'
FOLDER_LABELS = POSITIVES_FOLDER + 'detect/'
FOLDER_FRAMES = POSITIVES_FOLDER + 'frames_ms2/'

FIRST_FRAMES = FOLDER_FRAMES + '1606b0e6_0/'
SECOND_FRAMES = FOLDER_FRAMES + '1606b0e6_1/'

In [83]:
os.chdir(POSITIVES_FOLDER)
! pwd

/content/drive/MyDrive/Bundesliga/PositiveFrames


In [84]:
! mkdir crops

mkdir: cannot create directory ‘crops’: File exists


In [85]:
FIRST_FRAME_USED = 18

In [86]:
FOLDER_LABELS

'/content/drive/MyDrive/Bundesliga/PositiveFrames/detect/'

In [101]:
! mkdir crops/$video
# We iterate over every event, from 0 to 395
for event in range(EVENTS_PER_VIDEO[video]):
  # We establish ball = False (in this event, ball wasn't yet recognized)
  ball = False
  # We should find for example f'1606b0e6_0_{frame}_' and establish that class_ equals what follows it 
  substring = f'{FOLDER_LABELS}{video}/labels/{video}_{event}_'
  file = glob.glob(f'{substring}*.txt')[ANY_INDEX]
  len_subs = len(substring)
  class_ = int(file[len_subs])

  event_id = f'{video}_{event}_{class_}'

  # We create a condition based on the class on how to iterate for the frames.
  # Challenges' frame between tolerances are respectively: 17 yes, 1 yes, 17 yes.
  # For play and throw-in are respectively: 9 no, 8 yes, 1 yes, 8 yes, 9 no.

  # if class_ == 0: # if it's a challenge
    
  # TO AVOID BALL FPs: we iterate over every frames, from 18 to 34
  
  for frame in range(FIRST_FRAME_USED, FRAMES_PER_EVENT - 2): # - 2 cause we need to use 2 more after the one detected
    frame_id = f'{event_id}_{frame}' # e.g. '1606b0e6_0_0_0_0'
    print(frame_id)
    label_txt = f'{frame_id}.txt'
    with open(f'{FOLDER_LABELS}{video}/labels/{label_txt}') as f:
      reader = f.read()
    detections = [i.split(' ') for i in reader.split('\n')][:-1] # [:-1] because last element is always "''"
    # counter = 0

    for detection in detections: # in a single frame

      # confidences = list() # WE ARE NOT IMPLEMENTING CONFIDENCES FOR NOW AS IT MAY NOT BE RELEVANT
      if detection[CLASS_INDEX] == BALL_CLASS:
        # Ball is True for this event, it was detected in at least one of the frames.
        ball = True
        
        # confidence = float(detection[CONFIDENCE_INDEX])

        x = int(float(detection[X_INDEX]) * X_SIZE)
        y = int(float(detection[Y_INDEX]) * Y_SIZE)
        ymin = max(y - radius, 0)
        ymax = min(y + radius, Y_SIZE)
        xmin = max(x - radius, 0)
        xmax = min(x + radius, X_SIZE)

        # TO AVOID BALL FPs: we throw away detections that make the crop next to a limit of the frame
        if ymin == 0 or ymax == Y_SIZE or xmin == 0 or xmax == X_SIZE:
          ball = False

        # We make sure all crops are 224x224 (this won't apply if code above is implemented)
        # if ymin == 0:
        #   ymax = CROP_SIZE # = 224
        # elif ymax == Y_SIZE:
        #   ymin = Y_SIZE - CROP_SIZE # = 1024 - 224
        # if xmin == 0:
        #   xmax = CROP_SIZE # == 224 
        # elif xmax == X_SIZE:
        #   xmin = X_SIZE - CROP_SIZE # = 1928 - 224

        # break
        if ball: # NO IMAGE WILL BE CROPPED IF NO BALL IS SEEN BETWEEN FRAMES 18 AND 34

          for r in range(frame - 3, frame): # 3: amount of frames per event
            img = cv2.imread(f'{FOLDER_FRAMES}{video}/{event_id}_{r}.jpg')
            cropped_image = img[ymin:ymax, xmin:xmax] # We change from (x, y) to (y, x) to work with cv2_imshow
            cv2.imwrite(f'{POSITIVES_FOLDER}crops/{video}/{frame_id}_{r}cropped.jpg', cropped_image)
          break # breaks loop of detections in a single frame (so there are 3 per frame only)
    if ball:      
      break # breaks loop of frame in frames per event (so there are 3 per event only)

    elif frame == FRAMES_PER_EVENT - 3: # We implement crop on main frame (index 17) based on closest two people
      frame_id = f'{event_id}_{FIRST_FRAME_USED - 1}' # -1 so we get main fraim (index 17)
      label_txt = f'{frame_id}.txt'
      with open(f'{FOLDER_LABELS}{video}/labels/{label_txt}') as f:
        reader = f.read()
      detections = [i.split(' ') for i in reader.split('\n')][:-1] # [:-1] because last element is always "''"
      arr = np.array(detections)
      persons = arr[arr[:, 0] == '0']
      xy_persons = persons[:, X_INDEX:Y_INDEX + 1].astype(float)
      dists = cdist(xy_persons, xy_persons)
      dists[dists == 0] = dists.max()
      arg1, arg2 = np.unravel_index(dists.argmin(), dists.shape)
      two_people = xy_persons[[arg1, arg2]]
      frame_center = np.mean(two_people, axis=0)
      x, y = frame_center
      x = int(x * X_SIZE)
      y = int(y * Y_SIZE)
      ymin = max(y - radius, 0)
      ymax = min(y + radius, Y_SIZE)
      xmin = max(x - radius, 0)
      xmax = min(x + radius, X_SIZE)

      for r in range(FIRST_FRAME_USED - 1, FIRST_FRAME_USED + 2): # here we use -1 and + 2 because we are certain it's from 17 (our main frame)
        img = cv2.imread(f'{FOLDER_FRAMES}{video}/{event_id}_{r}.jpg')
        cropped_image = img[ymin:ymax, xmin:xmax] # We change from (x, y) to (y, x) to work with cv2_imshow
        cv2.imwrite(f'{POSITIVES_FOLDER}crops/{video}/{frame_id}_{r}cropped.jpg', cropped_image)
      break


mkdir: cannot create directory ‘crops/1606b0e6_0’: File exists
1606b0e6_0_0_0_18
1606b0e6_0_1_0_18
1606b0e6_0_2_1_18
1606b0e6_0_3_2_18
1606b0e6_0_3_2_19
1606b0e6_0_3_2_20
1606b0e6_0_4_2_18
1606b0e6_0_4_2_19
1606b0e6_0_5_2_18
1606b0e6_0_6_2_18
1606b0e6_0_7_2_18
1606b0e6_0_8_2_18
1606b0e6_0_8_2_19
1606b0e6_0_8_2_20
1606b0e6_0_8_2_21
1606b0e6_0_8_2_22
1606b0e6_0_8_2_23
1606b0e6_0_8_2_24
1606b0e6_0_9_2_18
1606b0e6_0_9_2_19
1606b0e6_0_9_2_20
1606b0e6_0_9_2_21
1606b0e6_0_10_2_18
1606b0e6_0_10_2_19
1606b0e6_0_10_2_20
1606b0e6_0_10_2_21
1606b0e6_0_10_2_22
1606b0e6_0_10_2_23
1606b0e6_0_10_2_24
1606b0e6_0_10_2_25
1606b0e6_0_11_2_18
1606b0e6_0_11_2_19
1606b0e6_0_11_2_20
1606b0e6_0_12_2_18
1606b0e6_0_13_2_18
1606b0e6_0_14_2_18
1606b0e6_0_14_2_19
1606b0e6_0_14_2_20
1606b0e6_0_15_2_18
1606b0e6_0_16_2_18
1606b0e6_0_17_2_18
1606b0e6_0_18_2_18
1606b0e6_0_18_2_19
1606b0e6_0_18_2_20
1606b0e6_0_18_2_21
1606b0e6_0_18_2_22
1606b0e6_0_19_2_18
1606b0e6_0_20_2_18
1606b0e6_0_21_2_18
1606b0e6_0_21_2_19
1606b0e6_

In [ ]:
POSITIVES_FOLDER = '/content/drive/MyDrive/Bundesliga/PositiveFrames/'
os.chdir(POSITIVES_FOLDER)

In [63]:
os.chdir('/content/drive/MyDrive/Bundesliga/')

In [102]:
! ls crops

1606b0e6_0


In [103]:
! ls crops/'1606b0e6_0'  | wc -l

1188


In [46]:
video_ids = ['1606b0e6_0', '1606b0e6_1',
             'cfbe2e94_0', 'cfbe2e94_0',
             'ecf251d4_0',
             '3c993bd2_0', '3c993bd2_1',
             '4ffd5986_0', 
             '9a97dae4_1', 
             '35bd9041_0', '35bd9041_1',
             '407c5a9e_1']

In [47]:
! pwd

/content/drive/MyDrive/Bundesliga/PositiveFrames


First time checked:

In [48]:
for video_id in video_ids:
  print(video_id)
  ! ls frames_ms2/$video_id | wc -l

1606b0e6_0
13860
1606b0e6_1
2426
cfbe2e94_0
3040
cfbe2e94_0
3040
ecf251d4_0
648
3c993bd2_0
659
3c993bd2_1
16036
4ffd5986_0
3050
9a97dae4_1
3091
35bd9041_0
636
35bd9041_1
587
407c5a9e_1
670


Second time checked:

In [105]:
# for video_id in video_ids:
#   print(video_id)
#   ! ls frames_ms2/$video_id | wc -l

In [ ]:
def get_positive_intervals(TABULAR_FILENAME):
    """ get intervals for events from tabular data (train.csv file) """
    
    df = pd.read_csv(TABULAR_FILENAME)
    df['time_ms'] = round(df['time'] * 1000, 0).astype(int)
    # get positive intervals
    df_starts = df[df['event']=='start'].copy()
    df_ends = df[df['event']=='end'].copy()

    df_positive_intervals = pd.DataFrame({'video_id': df_starts['video_id'].values,
                                        'start_time_ms': df_starts['time_ms'].values,
                                        'end_time_ms': df_ends['time_ms'].values})
    return df_positive_intervals
    

def get_frames_events(video_id, tabular_filename, videos_path, output_path, 
                      negatives=False, neg_size=None, npz=True, jpg=False, dim=None):
    """get frames of events for a given video (positive or negative classes),
       save npz and/or jpg files
       resize (optional)
       function takes:
            video_id
            tabular_filename: filename for tabular data
            videos_path: path to folder where the relevant video is located
            output_path: path to folder where npz and/or jpg files will be saved
            negatives: whether we are taking samples of negative class. 
                If True, we take only negatives, not positives. If False, we take only positives, not negatives.
            neg_size: size of sample of negatives
            npz: whether to save an npz file with results
            jpg: whether to save jpg files
            dim: dimensions of resized images (optional)
    """

    os.chdir(ROOT)
    # get times from tabular data
    if negatives:
        df_positive_intervals = get_positive_intervals(tabular_filename)
        times = get_neg_times(df_positive_intervals, video_id, neg_size)
    else:
        times = get_event_times(tabular_filename)
    n_frames = len(times)

    # capture video
    videoname = ''.join([video_id, '.mp4'])
    cap = cv2.VideoCapture(os.path.join(videos_path, videoname))

    # initialize result list
    res = []

    for i, time in enumerate(tqdm(times)):
        cap.set(cv2.CAP_PROP_POS_MSEC, time)  # move the time
        success, image = cap.read()
        if not success:
            print(f'{i+1} of {n_frames}, video_id: {video_id}, time: {time}, failed to read.')
        # resize
        if dim:
            image = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
        if npz:
            res.append(image)
        if jpg:
            cv2.imwrite(os.path.join(output_path, f'{video_id}_{time}.jpg'), image)
        del image
    cap.release()
    if npz:
        np.savez_compressed(os.path.join(output_path, ''.join([video_id, '.npz'])), np.array(res))

    if negatives:
        return times

We use persons:

In [184]:
# # We iterate over every event, from 0 to 395
# for event in range(EVENTS_PER_VIDEO[video]):
#   # We establish ball = False (in this event, ball wasn't yet recognized)
#   ball = False
#   # We should find for example f'1606b0e6_0_{frame}_' and establish y = what's following that 
#   substring = f'{FOLDER_LABELS}/{video}/labels/{video}_{event}_'
#   file = glob.glob(f'{substring}*.txt')[ANY_INDEX]
#   len_subs = len(substring)
#   y = int(file[len_subs])
#   # We create a condition based on the class on how to iterate for the frames.
#   # Challenges' frame between tolerances are respectively: 17 yes, 1 yes, 17 yes.
#   # For play and throw-in are respectively: 9 no, 8 yes, 1 yes, 8 yes, 9 no.
#   if y == 0: # if it's a challenge

#   # We iterate over every frame, from 0 to 34
#   for frame in range(FRAMES_PER_EVENT):
#     identifier = f'{video}_{event}_{y}_{frame}' # e.g. '1606b0e6_0_0_0_0'
#     label_txt = f'{identifier}.txt'
#     with open(f'{FOLDER_LABELS}{video}/labels/{label_txt}') as f:
#       reader = f.read()
#     objects = [i.split(' ') for i in reader.split('\n')]
#     print(identifier)
#     counter = 0

#     for object_ in objects:
#       # confidences = list() # WE ARE NOT IMPLEMENTING CONFIDENCES FOR NOW AS IT MAY NOT BE RELEVANT
#       if object_[CLASS_INDEX] == BALL_CLASS:
#         # Ball is True for this event, it was detected in at least one of the frames.
#         ball = True
#         # confidence = float(object_[CONFIDENCE_INDEX])
#         x = int(float(object_[X_INDEX]) * X_SIZE)
#         y = int(float(object_[Y_INDEX]) * Y_SIZE)
#         ymin = max(y - radius, 0)
#         ymax = min(y + radius, Y_SIZE)
#         xmin = max(x - radius, 0)
#         xmax = min(x + radius, X_SIZE)

#         # TO AVOID FPs: we throw away detections that make the crop next to a limit of the frame
#         if ymin == 0 or ymax == Y_SIZE or xmin == 0 or xmax = X_SIZE:
#           ball = False

#         # We make sure all crops are 224x224 (this won't apply if code above is implemented)
#         # if ymin == 0:
#         #   ymax = CROP_SIZE # = 224
#         # elif ymax == Y_SIZE:
#         #   ymin = Y_SIZE - CROP_SIZE # = 1024 - 224
#         # if xmin == 0:
#         #   xmax = CROP_SIZE # == 224 
#         # elif xmax == X_SIZE:
#         #   xmin = X_SIZE - CROP_SIZE # = 1928 - 224

#         if ball:
#           img = cv2.imread(frame + '.jpg')
#           cropped_image = img[ymin:ymax, xmin:xmax] # We change from (x, y) to (y, x) to work with cv2_imshow
#           cv2.imwrite(f'{FOLDER_FRAMES}{frame}_cropped.jpg', cropped_image)
      
#     if not ball and USE_PERSONS:
#       counter += 1
#     if counter == USE_PERSONS_THRESH: 
#       # Maybe create a counter += 1, and when the ball is not detected in 15? consecutive frames, generate the location like this:
#       FIRST_FILENAME_INDEX = 1
#       label_txt = f'{video}_{FIRST_FILENAME_INDEX}.txt'
#       with open(f'{FOLDER_LABELS}{label_txt}') as f: # Actually the reading should have happened before, so I may have to check this.
#         reader = f.read()
#       objects = [i.split(' ') for i in reader.split('\n')][:-1]
#       arr = np.array(objects)
#       persons = arr[arr[:, 0] == '0']
#       xy_persons = persons[:, X_INDEX:Y_INDEX + 1].astype(float)
#       dists = cdist(xy_persons, xy_persons)
#       dists[dists == 0] = dists.max()
#       arg1, arg2 = np.unravel_index(dists.argmin(), dists.shape)
#       two_people = xy_persons[[arg1, arg2]]
#       frame_center = np.mean(two_people, axis=0)

1606b0e6_0_0_0_0
1606b0e6_0_0_0_1


TypeError: ignored

In [21]:
# for i, frame in enumerate(filenames, 1): # frame of video (filenames contains frames of a single video)
#   label_txt = f'{video}_{i}.txt'
#   with open(f'{FOLDER_LABELS}{label_txt}') as f:
#     reader = f.read()
#   objects = [i.split(' ') for i in reader.split('\n')]
#   ball = False
#   counter = 0

#   for object_ in objects:
#     if object_[CLASS_INDEX] == BALL_CLASS:
#       ball = True
#       x = int(float(object_[X_INDEX]) * X_SIZE)
#       y = int(float(object_[Y_INDEX]) * Y_SIZE)
#       ymin = max(y - radius, 0)
#       ymax = min(y + radius, Y_SIZE)
#       xmin = max(x - radius, 0)
#       xmax = min(x + radius, X_SIZE)

#       img = cv2.imread(frame + '.jpg') # IT CROPS MORE THAN 1 WHEN 2 OR MORE BALLS ARE IN A FRAME, BUT IT OVERWRITES IT
#       cropped_image = img[ymin:ymax, xmin:xmax] # We change from (x, y) to (y, x) to work with cv2_imshow
#       cv2.imwrite(f'{FOLDER_FRAMES}{frame}_cropped.jpg', cropped_image)
    
#   if not ball and USE_PERSONS:
#     counter += 1
#   if counter == USE_PERSONS_THRESH: 
#     # Maybe create a counter += 1, and when the ball is not detected in 15? consecutive frames, generate the location like this:
#     FIRST_FILENAME_INDEX = 1
#     label_txt = f'{video}_{FIRST_FILENAME_INDEX}.txt'
#     with open(f'{FOLDER_LABELS}{label_txt}') as f: # Actually the reading should have happened before, so I may have to check this.
#       reader = f.read()
#     objects = [i.split(' ') for i in reader.split('\n')][:-1]
#     arr = np.array(objects)
#     persons = arr[arr[:, 0] == '0']
#     xy_persons = persons[:, X_INDEX:Y_INDEX + 1].astype(float)
#     dists = cdist(xy_persons, xy_persons)
#     dists[dists == 0] = dists.max()
#     arg1, arg2 = np.unravel_index(dists.argmin(), dists.shape)
#     two_people = xy_persons[[arg1, arg2]]
#     frame_center = np.mean(two_people, axis=0)

NameError: ignored